In [ ]:
# this file need few corrections on the few last cells [11, 12, 13]


import re
import string
import nltk
import regex as re 
import pandas as pd 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import PunktSentenceTokenizer, word_tokenize

In [2]:
file_path = r"C:\Users\Sujal Karmakar\Desktop\Desktop\Data Analyst\Python\python_data_analytics_project\Theme Finder Using Caption (NLP)\Data\leanbeast_analysis_ready.csv"

df = pd.read_csv(file_path)

#### 1) Basic Cleaning (lower case, remover numbers, emoji, etc)

In [3]:
df_caption = df["caption_text"]

In [4]:
# Function to clean each caption: lowercase, remove punctuation, then remove emojis
def clean_caption(text):
    if pd.isna(text):
        return ""
    text = text.lower()  # Lowercase

    text = re.sub(r"\p{P}+", "", text)

    # Remove punctuation
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)

    # Remove emojis
    emoji_pattern = re.compile(
        "["
        "\U0001f600-\U0001f64f"  # emoticons
        "\U0001f300-\U0001f5ff"  # symbols & pictographs
        "\U0001f680-\U0001f6ff"  # transport & map symbols
        "\U0001f1e0-\U0001f1ff"  # flags (iOS)
        "\U00002700-\U000027bf"  # Dingbats
        "\U0001f900-\U0001f9ff"  # Supplemental Symbols and Pictographs
        "\U00002600-\U000026ff"  # Misc symbols
        "\U00002b00-\U00002bff"  # Misc symbols and arrows
        "\U0001fa70-\U0001faff"  # Symbols and Pictographs Extended-A
        "\ufe0f"  # Variation Selector-16 (emoji style)
        "]+",
        flags=re.UNICODE,
    )
    text = emoji_pattern.sub(r"", text)
    return text


# Apply to the caption column
df_caption_cleaned = df_caption.apply(clean_caption)

In [5]:
df_caption_cleaned.sample(15)

1244    gaining tips  abs fitness gym workout bodybuil...
212     comment send and i will send you a sample six ...
429      cholesterol is not ur enemy stop getting fooled 
1072                         worlds smallest bodybuilder 
1053                                      sunday gym vlog
1112    apni asli aukaat khud pehchano people will not...
267                       who all does icebath in winters
795                                        asli veg khana
480                                            chai sugar
221     comment healthy and i will message you 5 healt...
1044                                              mykonos
1323    how a perfect fatloss meal looks   abs fitness...
651     couples who exercise togetherstays together  f...
852                         high protein zero carbs pizza
604     bss itnaa khalokaafi he fitwithiqbal instagram...
Name: caption_text, dtype: object

#### 2) Tokenization (removing stop words)

In [6]:
# import nltk

# # Define a custom download directory
# custom_nltk_path = r'C:\Users\Sujal Karmakar\.conda\envs\DS\nltk_data'

# # Download resources to the custom path
## nltk.download('punkt', download_dir=custom_nltk_path)
# nltk.download('punkt_tab', download_dir=custom_nltk_path)
# nltk.download('stopwords', download_dir=custom_nltk_path)

# # Manually add the path to NLTK's search locations
# nltk.data.path.append(custom_nltk_path)

# i had to explicitly download these by specifying files and also punkt was not working for me nltk was demanding punkt_tab so i did that 


In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

# Set of English stopwords for filtering
stop_words = set(stopwords.words("english"))


def preprocess_caption(text):
    # Handle non-string inputs safely
    if not isinstance(text, str):
        return []

    # Step 1: Sentence tokenize the text
    sentences = sent_tokenize(text)

    # Step 2: Word tokenize each sentence
    tokens = [word_tokenize(sentence) for sentence in sentences]

    # Step 3: Flatten the list of token lists into a single list
    flat_tokens = [token for sublist in tokens for token in sublist]

    # Step 4: Remove stopwords (case-insensitive)
    filtered_tokens = [w for w in flat_tokens if w.lower() not in stop_words]

    return filtered_tokens


df_caption_cleaned_tokens = df_caption_cleaned.apply(preprocess_caption)

# Show first few results
print(df_caption_cleaned_tokens.head())

0    [well, done, jaibajaj0786, lost, 39kgs, withou...
1    [transformation, alert, keen, interested, paid...
2    [shredded, mode, manavkansagra1111, guidance, ...
3    [comment, atta, send, link, healthy, atta, mes...
4    [dinner, date, couple, love, couplegoals, wedd...
Name: caption_text, dtype: object


In [8]:
df_caption_combined = pd.DataFrame({
    "cleaned_caption": df_caption_cleaned,
    "tokens": df_caption_cleaned_tokens
})
df_caption_cleaned

0       well done jaibajaj0786 lost 39kgs without coun...
1       transformation alert   if he keen do it  why n...
2       shredded mode on manavkansagra1111 under my gu...
3       comment atta and i will send you the link of h...
4       dinner date   couple love couplegoals wedding ...
                              ...                        
1546    lockdown transformation lost 15 kgs in 70 days...
1547    shoot mode  modelifefitness gym workout motiva...
1548    manage your life  office and work  paid online...
1549    throwback to one my client who reduced 38 kgs ...
1550    taj mahal is beautiful today gymlife training ...
Name: caption_text, Length: 1551, dtype: object

#### 3) Creating dictionaries and corpus for LDA (Linear discriminant analysis)

In [9]:
import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel

# Convert the Series of lists into a list of lists
texts = df_caption_cleaned_tokens.tolist()

# Create Dictionary: maps each word to a unique id
dictionary = corpora.Dictionary(texts)

# Filter extremes to remove very rare and very common words (optional but recommended)
dictionary.filter_extremes(no_below=5, no_above=0.5)  # keeps words in at least 5 docs but less than 50% of all docs

# Create Corpus: list of bags of words for each document
corpus = [dictionary.doc2bow(text) for text in texts]

print(len(texts))
print(len(corpus))
# print(f"Number of unique tokens: {len(dictionary)}")
# print(f"Number of documents: {len(corpus)}")
# print(f"Sample doc (bow format): {corpus[0]}")

# Number of topics LDA will find 
num_topics = 10

# Build LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=42, passes=10)

# Show the top words for each topic
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic {idx}: {topic}")



1551
1551
Topic 0: 0.039*"love" + 0.034*"wedding" + 0.034*"instagram" + 0.033*"happy" + 0.033*"like" + 0.033*"photography" + 0.033*"instagood" + 0.030*"photooftheday" + 0.029*"beautiful" + 0.028*"follow"
Topic 1: 0.070*"comment" + 0.049*"send" + 0.036*"sugar" + 0.022*"healthy" + 0.022*"inbox" + 0.018*"link" + 0.018*"diet" + 0.014*"fasting" + 0.013*"meal" + 0.013*"high"
Topic 2: 0.019*"fitness" + 0.019*"lifestyle" + 0.018*"muscle" + 0.018*"fit" + 0.017*"fitnessmotivation" + 0.017*"training" + 0.015*"gym" + 0.015*"zero" + 0.014*"motivation" + 0.014*"gymmotivation"
Topic 3: 0.081*"hai" + 0.035*"meh" + 0.032*"nahi" + 0.024*"se" + 0.024*"kya" + 0.023*"body" + 0.020*"weight" + 0.019*"ka" + 0.019*"ki" + 0.016*"ye"
Topic 4: 0.044*"protein" + 0.034*"food" + 0.029*"u" + 0.026*"whey" + 0.023*"gym" + 0.016*"best" + 0.015*"per" + 0.014*"ur" + 0.013*"get" + 0.012*"egg"
Topic 5: 0.034*"gym" + 0.031*"workout" + 0.031*"bodybuilding" + 0.031*"abs" + 0.031*"body" + 0.031*"fitness" + 0.031*"cardio" + 0.03

#### Topic names as per captions.

| **Topic #** | **Top Keywords**                             | **Final Topic Name**         | **Why this name fits**                                       |
| ----------- | -------------------------------------------- | ---------------------------- | ------------------------------------------------------------ |
| **0**       | love, instagram, happy, wedding, photography | `Personal & Lifestyle`       | Captions about daily life, emotions, events (e.g., weddings) |
| **1**       | sugar, comment, send, healthy, inbox         | `Diet Tips`                  | Sugar, healthy advice, often CTA-based captions              |
| **2**       | muscle, gym, workout, gain, fit              | `Muscle Building & Gym`      | Bodybuilding and gym-focused motivational content            |
| **3**       | hai, nahi, se, kya, cheese, meal             | `Meal Talks`                 | Conversational tone, mostly food or meal-related             |
| **4**       | best, protein, source, india, egg            | `Protein Sources in Diet`    | Specific food items as sources of protein                    |
| **5**       | gym, hai, fitness, bhai, shredded            | `Home Gym & Motivation`      | Desi, motivational gym talk (often home setups)              |
| **6**       | healthy, fitness, training, healthylifestyle | `Fitness Lifestyle`          | General health, fitness, and lifestyle-focused tips          |
| **7**       | protein, veg, comment, paneer, recipe        | `Vegetarian Protein Recipes` | Vegetarian food-based protein suggestions & recipes          |
| **8**       | acid, uric, animal, body, muscle             | `Internal Body Science`      | Concepts like uric acid, testosterone, internal processes    |
| **9**       | icebath, creatine, bath, calorie             | `Recovery & Supplements`     | Post-gym recovery, supplements, and routines like ice baths  |
| **10**      | workout, gym, abs, muscle, gains             | `Workout Tips`               | Focused on physical training, routines, gym motivation       |
| **11**      | transformation, whatsapp, paid, diet         | `Client Results & Services`  | Paid plans, before-after results, WhatsApp CTAs              |
| **12**      | body, weight, mass, energy, lean             | `Body Composition & Weight Tips`  | Weight gain/loss, lean muscle, energy balance                |
| **13**      | cholesterol, fasting, insulin, comment       | `Health Science`             | Scientific nutrition (cholesterol, fasting, insulin)         |
| **14**      | heart, attack, gym, look, lets               | `Heart Health & Awareness`   | Heart care, awareness, lifestyle triggers                    |


In [10]:
dominant_topics = []
dominant_topic_probabilities = []

for bow in corpus:
    if len(bow) == 0:
        # Empty bow → assign 'Other'
        dominant_topics.append("Other")
        dominant_topic_probabilities.append(1.0)
    else:
        topic_probs = lda_model.get_document_topics(bow)
        if len(topic_probs) == 0:
            # Just in case, fallback to 'Other'
            dominant_topics.append("Other")
            dominant_topic_probabilities.append(1.0)
        else:
            # Pick topic with highest probability
            dominant_topic, max_prob = max(topic_probs, key=lambda x: x[1])
            dominant_topics.append(dominant_topic)
            dominant_topic_probabilities.append(max_prob)

# Add to your original df_caption_cleaned_tokens or main DataFrame (same length)
df_caption_cleaned_tokens = df_caption_cleaned_tokens.reset_index(drop=True)  # to be safe

df_caption_cleaned_tokens = pd.DataFrame(df_caption_cleaned_tokens)  # if not already a DataFrame

df_caption_cleaned_tokens['Dominant_Topic'] = dominant_topics
df_caption_cleaned_tokens['Dominant_Topic_Prob'] = dominant_topic_probabilities


In [11]:
# Topic name mapping
topic_name_mapping = {
    0: "Personal & Lifestyle",
    1: "Diet Tips",
    2: "Muscle Building & Gym",
    3: "Meal Talks",
    4: "Protein Sources in Diet",
    5: "Home Gym & Motivation",
    6: "Fitness Lifestyle",
    7: "Healthy Recipes",
    8: "Internal Body Science",
    9: "Recovery & Supplements",
    10: "Workout Tips",
    11: "Client Results & Services",
    12: "Body Composition & Weight Tips",
    13: "Health Science",
    14: "Heart Health & Awareness",
    "Other": "Other / Empty Caption"
}

# Apply to the cleaned tokens DataFrame
df_caption_cleaned_tokens["Topic_Name"] = df_caption_cleaned_tokens["Dominant_Topic"].map(topic_name_mapping)


In [12]:
# Reset and clean both DataFrames
df_caption = df_caption.reset_index(drop=True)
df_topic_info = df_caption_cleaned_tokens[["Dominant_Topic", "Dominant_Topic_Prob", "Topic_Name"]]

# Join the topic info with original captions
df_with_topics = df_caption.to_frame(name="Original_Caption").join(df_topic_info)



In [13]:
# Show sample captions for each topic
for topic in df_with_topics["Topic_Name"].unique():
    print(f"\n🔹 Topic: {topic}")
    sample = df_with_topics[df_with_topics["Topic_Name"] == topic].sample(5)
    for i, row in sample.iterrows():
        print(f" - {row['Original_Caption']}")



🔹 Topic: Meal Talks
 - Body meh investment se accha investment aur koi nahi kyo:- 1. App healthy rehte ho 2. 4-5 log tariff kar dete hai. 3. Personal and professional life meh aap taraki karte ho. 4 . Will power strong hoti hai. 5. Kabhi haar nahi maante 6. Tabiyat kam kharab hoti hai
 - Muscle gain karne meh kitna time lagta hai
 - Natural vs steroids body diff
 - Turned 36 years young❤️
 - Gym karne ki umar nikal gai hai

🔹 Topic: Healthy Recipes
 - Nakli paneer pakda gaya☠️
 - Comment “Eggs” and i will send the recipe of egg bhurji .  ✅Interested in Paid transformation?? Message me now or Whatsapp me on +919171123324 ✅
 - Comment, “avocado” and I will send you the recipe of amazing avocado spread.
 - Common “fasting” and I will send you a blueprint of how to start fasting and implement it forever.  ✅Interested in Paid transformation?? Message me now or Whatsapp me on +919171123324 ✅
 - Comment “diet” to know about his diet’s blueprint  ✅Interested in Paid transformation?? Message m